In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

## Классы для расчёта 2D дискрипторов

In [3]:
class DescriptionCalc:
    def __init__(self, smiles):
        '''
        Класс расчитывает физико-хмические признаки молекулы, которые описывают молекулу

        smiles - список веществ 
        '''
        self.mols = []
        for i in tqdm(smiles):
            try:
                self.mols.append(Chem.MolFromSmiles(i))
            except:
                self.mols.append(None)
                
        self.smiles = smiles

class RDKit_2D(DescriptionCalc):
    def __init__(self, smiles):
        super().__init__(smiles)

    def compute_2Drdkit(self, name):
        rdkit_2d_desc = []
        calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
        header = calc.GetDescriptorNames()
        
        def calculate_descriptors(mol):
            if mol is not None:
                return calc.CalcDescriptors(mol)
            else:
                return [None] * len(header)
            
        rdkit_2d_desc = [calculate_descriptors(mol) for mol in tqdm(self.mols)]
        
        df = pd.DataFrame(rdkit_2d_desc, columns=header)
        df.insert(loc=0, column= name, value=self.smiles)
        return df
    
class MACCS(DescriptionCalc):
    def __init__(self, smiles):
        super().__init__(smiles)

    def compute_MACCS(self, name):
        MACCS_list = []
        header = ['bit' + str(i) for i in range(167)]
        
        def calculate_descriptors(mol):
            if mol != None:
                return list(MACCSkeys.GenMACCSKeys(mol).ToBitString())
            else:
                return [0 for i in range(167)]

        MACCS_list = [calculate_descriptors(mol) for mol in tqdm(self.mols)]
        
        df = pd.DataFrame(MACCS_list,columns=header)
        df.insert(loc=0, column= name, value=self.smiles)
        return df

class ECFP6(DescriptionCalc):
    def __init__(self, smiles):
        super().__init__(smiles)

    def mol2fp(self, mol, radius = 3):
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius = radius)
        return fp

    def compute_ECFP6(self, name):
        bit_headers = ['bit' + str(i) for i in range(2048)]
        arr = lil_matrix((len(self.mols), 2048), dtype=np.int8)
        
        for idx, mol in enumerate(tqdm(self.mols)):
            if mol != None:
                fp = self.mol2fp(mol)
                on_bits = fp.GetOnBits()

                for bit in on_bits:
                    arr[idx, bit] = 1
                    
        df_ecfp6 = pd.DataFrame.sparse.from_spmatrix(arr, columns=bit_headers)
        df_ecfp6.insert(loc=0, column= name, value=self.smiles)
        return df_ecfp6
    
def main():
    exsample_smiles = ['C1CCCCC1', 'c1ccccc1']

    ecfp6 = ECFP6(exsample_smiles)
    res = ecfp6.compute_ECFP6('exsample')
    print(res)

    rdkit_2d = RDKit_2D(exsample_smiles)
    res = rdkit_2d.compute_2Drdkit('exsample')
    print(res)

    maccs = MACCS(exsample_smiles)
    res = maccs.compute_MACCS('exsample')
    print(res)
    
    
if __name__ == '__main__':
    main()

100%|██████████| 2/2 [00:00<00:00, 4688.99it/s]


   exsample  bit0  bit1  bit2  ...  bit2044  bit2045  bit2046  bit2047
0  C1CCCCC1     0     0     1  ...        0        0        0        0
1  c1ccccc1     0     0     0  ...        0        0        0        0

[2 rows x 2049 columns]


100%|██████████| 2/2 [00:00<00:00, 252.87it/s]


   exsample  MaxAbsEStateIndex  ...  fr_unbrch_alkane  fr_urea
0  C1CCCCC1                1.5  ...                 0        0
1  c1ccccc1                2.0  ...                 0        0

[2 rows x 211 columns]


100%|██████████| 2/2 [00:00<00:00, 1225.15it/s]

   exsample bit0 bit1 bit2 bit3 bit4  ... bit161 bit162 bit163 bit164 bit165 bit166
0  C1CCCCC1    0    0    0    0    0  ...      0      0      1      0      1      0
1  c1ccccc1    0    0    0    0    0  ...      0      1      1      0      1      0

[2 rows x 168 columns]


## Загружаем молекулы для расчёта

In [4]:
df = pd.read_table('/home/jupyter/datasphere/project/Расчёт дискрипторов/mapper_moleculares.tsv')
df.head()

,mol,prepare_mol
0,O=[N+:22]([O-])[C:25]1=[C:26]([Cl:34])[C:27]([...,O=[N+]([O-])[C]1=[C]([Cl])[C]([Cl])=N[C]([Cl])...
1,C[Si-](C)(C)(F)F,C[Si-](C)(C)(F)F
2,OC(=O)C1=CN2C=C(C(F)(F)F)[CH:19]=[C:14](Cl)[C:...,OC(=O)C1=CN2C=C(C(F)(F)F)[CH]=[C](Cl)[C]2=N1
3,CC1(C)[N:19]([C:24]([CH:25]([F:26])[F:27])=[O:...,CC1(C)[N]([C]([CH]([F])[F])=[O])[C@H]([CH2][F]...
4,[CH2:1]([CH2:2][CH2:3][CH2:4][CH2:5][CH3:6])[C...,[CH2]([CH2][CH2][CH2][CH2][CH3])[C]1=[C]([CH]=...


Считаем дискрипторы для каждой канонизированной молекулы

In [5]:
smiles = list(df.prepare_mol.unique())

In [6]:
ecfp6 = ECFP6(smiles)
res_ecfp6 = ecfp6.compute_ECFP6('smiles')

 15%|█▌        | 131225/870765 [00:12<01:10, 10552.65it/s]


KeyboardInterrupt: 

In [6]:
res_ecfp6.to_pickle('ecfp6_feature.pickle')

In [7]:
rdkit_2d = RDKit_2D(smiles)
res_rdkit_2d = rdkit_2d.compute_2Drdkit('smiles')

 36%|███▋      | 316181/870765 [42:44<1:13:54, 125.06it/s]Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/home/jupyter/.local/lib/python3.10/site-packages/rdkit/Chem/SpacialScore.py", line 72, in SPS
    return _SpacialScore(mol, normalize=normalize).score
  File "/home/jupyter/.local/lib/python3.10/site-packages/rdkit/Chem/SpacialScore.py", line 95, in __init__
    self.score /= self.mol.GetNumHeavyAtoms()
ZeroDivisionError: division by zero
 85%|████████▌ | 742075/870765 [1:40:53<17:00, 126.14it/s]  Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/home/jupyter/.local/lib/python3.10/site-packages/rdkit/Chem/SpacialScore.py", line 72, in SPS
    return _SpacialScore(mol, normalize=normalize).sc

In [8]:
res_rdkit_2d.to_pickle('rdkit_2d_feature.pickle')

In [7]:
maccs = MACCS(smiles)
res_maccs = maccs.compute_MACCS('smiles')

100%|██████████| 870765/870765 [07:12<00:00, 2013.19it/s]


In [8]:
res_maccs.to_pickle('maccs_feature.pickle')